**Imported Packages and Datasets**

In [1]:
import pandas as pd
from zipfile import ZipFile
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython import display
from time import sleep
import scipy.stats  as stats
from scipy.stats import pearsonr
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import math
import sklearn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from itertools import permutations
import itertools
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
# Import Literacy and numeracy dataset, define list of countries, years and features in dataset
File = pd.read_csv('LiteracyNumeracyDataset2.csv')
country = 'Country'
countries = File.Country.unique()
Year = ['2013','2014','2015','2016','2017','2018']
Indicator = File.Indicator.unique()

In [3]:
# Import political gender balance dataset, and defining features and years in dataset
FileGen = pd.read_csv('PoliticalGenderBalanceDataset2.csv')
Year = ['2013','2014','2015','2016','2017','2018']
IndicatorGen = FileGen.Indicator.unique()

In [4]:
# Import unicef dataset 1 (Financial inclusion, family demand, labour participation rates)
FileUnicef1 = pd.read_csv('UnicefDatasets1.csv',encoding = "ISO-8859-1")
IndicatorUnicef1 = list(FileUnicef1.columns.values[3:])

**Functions**

In [5]:
# function to rearrange dataframe
def rearrange_dataframe(df, indicator_name):
    #years = [c for c in df.columns if c[0] == '1' or c[0] == '2']
    years = Year
    df = df.loc[df['Indicator'] == indicator_name]
    df = pd.melt(df[[country] + years], id_vars=country, var_name='year')
    ## https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html
    df.rename(columns={'value': indicator_name}, inplace=True)
    df.set_index(['year', country], inplace=True)
    return df

# function to get most recent value of attribute of each country and create dataset
def get_recent_value(df,indicator_name):
    data = pd.DataFrame(columns = ['Country',indicator_name])
    years = Year
    df = df.loc[df['Indicator'] == indicator_name]
    #List = []
    for i in range(len(countries)):
        ValueList = []
        num = 0
        ValueList.append(countries[i])
        ValueList.append(np.nan)
        #ValueList.append('Nan')
        for j in range(len(years)-1,-1,-1):
            q = df[df['Country'] == countries[i]]
            a = q.loc[:,years[j]]
            #if (str(a.item())!='nan') and (num == 0):
            if (str(a.item()) != str(np.nan)) and (num == 0):
                ValueList[1] = float(a.item())
                num = 1
        data = data.append({'Country': ValueList[0], indicator_name: ValueList[1]}, ignore_index=True)
    return data

# function to find correlations and significance between features of dataset
def Correlation_features(X,IndicatorList):
    CorrMatrix = np.zeros((len(IndicatorList),len(IndicatorList)))
    SigMatrix = np.zeros((len(IndicatorList),len(IndicatorList)))
    for i in range(len(IndicatorList)):
        FirstInd = X[:,i]
        for j in range(len(IndicatorList)):
            if i == j:
                CorrMatrix[i,j] = 1
                continue
            else:
                SecondInd = X[:,j]
                # Correlation
                correlation = pearsonr(FirstInd,SecondInd)
                CorrMatrix[i,j] = correlation[0]
                # Significance
                SigMatrix[i,j] = correlation[1]
    return CorrMatrix, SigMatrix

# function that prints heatmap of correlations
def correlation_heatmap(CorrMatrix):
    %matplotlib qt
    ax = sns.heatmap(CorrMatrix, vmin=-1, vmax=1,cmap='cool')
    return

# function that prints moderate correlations that are significant
def significant_correlations(CorrMatrix,SigMatrix,IndicatorList):
    ModulusMatrix = abs(CorrMatrix)
    a = 0
    print("\033[1m" + "Indicators with a moderate correlation (>0.5) and are significant at a 5% level of significance:"
          + "\033[0;0m")
    for i in range(len(IndicatorList)):
        for j in range(len(IndicatorList)):
            if i == j:
                continue
            if (ModulusMatrix[i,j] > 0.5) and (SigMatrix[i,j] < 0.05):
                a = a + 1
                print(str(IndicatorList[i]) + " V.S. " + str(IndicatorList[j]) + " ---> " + str(CorrMatrix[i,j]) + "\n")
    if a == 0:
        print("None")
    return

# function that plots indicator over time
def plot_indicator(df, indicator_name):
    fig, ax = plt.subplots(figsize=[15 ,10])
    for label, dfi in df.groupby(level=1):
        dfi[indicator].plot(ax=ax, label=label)
    plt.legend()
    ax.set_ylabel(indicator)
    ax.set_xticklabels(df1c.index.levels[0].values)
    ax.set_xlabel('year')
    return

# function that assigns a group colour to each datapoint based on label (GNI or Region)
def get_regions_and_gni(File,r,g,cl,attribute):
    RegionsList = []
    REGIONS = []
    gniList = []
    GNI = []
    colourList = []
    COLOUR = []
    for k in range(len(countries)):
        Data = File.loc[File['Country'] == countries[k]]
        RegionName = Data['Region'].unique()
        gniName = Data['GNI'].unique()
        RegionsList.extend(RegionName)
        gniList.extend(gniName)
        for i in range(len(r)):
            if (str(r[i]) == str(RegionName)) and (attribute == 'Region'):
                colourList.append(cl[i])
            if i < len(g):
                if (str(g[i]) == str(gniName)) and (attribute == 'GNI'):
                    colourList.append(cl[i])
    return RegionsList,colourList,gniList

# function that plots a 3d scatterplot of chosen features
def plot_scatter(xvalues,yvalues,zvalues,REGIONS,GNI,attribute,Index1,Index2,Index3):
    %matplotlib qt
    fig = plt.figure(num=1)
    ax = fig.add_subplot(111, projection='3d')
    if (attribute == 'Region'):
        for j in range(len(r)):
            index = []
            index = [i for i, x in enumerate(REGIONS) if x == r[j]]
            xx = [xvalues[x] for x in index]
            yy = [yvalues[x] for x in index]
            zz = [zvalues[x] for x in index]
            ax.scatter(xx,yy,zz,label=r[j],color=cl[j])
            ax.legend(loc='best')
            #ax.set_xlabel(IndicatorList[Index1])
            #ax.set_ylabel(IndicatorList[Index2])
            #ax.set_zlabel(IndicatorList[Index3])
    if (attribute == 'GNI'):
        for j in range(len(g)):
            index = []
            index = [i for i, x in enumerate(GNI) if x == g[j]]
            xx = [xvalues[x] for x in index]
            yy = [yvalues[x] for x in index]
            zz = [zvalues[x] for x in index]
            ax.scatter(xx,yy,zz,label=g[j],color=cl[j])
            ax.legend(loc='best')
            ax.set_xlabel(IndicatorList[Index1])
            ax.set_ylabel(IndicatorList[Index2])
            ax.set_zlabel(IndicatorList[Index3])

# function that finds the most likely mapping of K-means labels to GNI or Region labels in dataset (thus to test accuracy
# of K-means in classifying "unseen" test datapoints)
def permutation_labels(q,attribute,Klabels,y_train):
    per = list(permutations(range(len(q)))) 
    correctlabelslist = np.zeros((len(per)))
    for i in range(len(per)):
        labeldict = {per[i][c]:q[c] for c in range(len(q))}
        KLabelPer = [labeldict.get(item,item)  for item in Klabels]
        BothLabels = np.column_stack([KLabelPer,y_train]) 
        BothLabelsCompare = [BothLabels[c][0] == BothLabels[c][1] for c in range(len(BothLabels))]
        CorrectLabels = (float(BothLabelsCompare.count(True))/float(len(y_train)))*100
        correctlabelslist[i] = CorrectLabels
    maxcorrect = np.argmax(correctlabelslist)
    per = per[maxcorrect]
    return per

# function that finds accuracy of k-means in classifying test datapoints
def K_means_test_accuracy(num,t_size,q,attribute,OUTPUT,xvalues,yvalues,zvalues,TestAccuracyMean,b,c):
    testaccuracy = np.zeros((1,num))
    for i in range(len(testaccuracy[0])):
        Xinput = np.column_stack([np.array(xvalues),np.array(yvalues),np.array(zvalues)])
        X_train, X_test, y_train, y_test = train_test_split(Xinput, OUTPUT, test_size=t_size, random_state=i)
        kmeans = KMeans(n_clusters=int(len(q)),random_state = c).fit(X_train)
        Klabels = kmeans.labels_
        per = permutation_labels(q,attribute,Klabels,y_train)
        labeldict = {per[c]:q[c] for c in range(len(q))}
        y_pred = kmeans.predict(X_test)
        y_pred = [labeldict.get(item,item)  for item in y_pred]
        BothLabelstest = np.column_stack([y_pred,y_test]) 
        BothLabelsComparetest = [BothLabelstest[c][0] == BothLabelstest[c][1] for c in range(0,len(BothLabelstest))]
        CorrectLabelstest = (float(BothLabelsComparetest.count(True))/float(len(y_test)))*100
        testaccuracy[0,i] = CorrectLabelstest
    TestAccuracyMean[0,b] = np.mean(testaccuracy)
    return TestAccuracyMean

**Rearranged and merged datasets (using random forest to predict missing values)**

In [6]:
# REARRANGE DATASETS IN ORDER TO USE RANDOM FOREST TO PREDICT MISSING VALUES OF DATASET

# Rearrange all Literacy Data
LitData = get_recent_value(File,Indicator[0])
for i in range(1,len(Indicator)):
    LitCol = get_recent_value(File,Indicator[i])
    LitData = LitData.merge(LitCol, left_index=True, right_index=True)
LitData = LitData.iloc[:,1::2]

# Rearrange all Politics Data
PolData = get_recent_value(FileGen,IndicatorGen[0])
for i in range(1,len(IndicatorGen)):
    PolCol = get_recent_value(FileGen,IndicatorGen[i])
    PolData = PolData.merge(PolCol, left_index=True, right_index=True)
PolData = PolData.iloc[:,1::2]

# Rearrange data in UnicefDatasets1
Uni1Data = FileUnicef1.loc[FileUnicef1['Country'].isin(countries)]
Uni1Data = Uni1Data.reset_index(inplace = False) 
Uni1Data = Uni1Data.iloc[:,4:]

# merge all data together
AllData = LitData.merge(PolData,left_index=True, right_index=True)
AllData = AllData.merge(Uni1Data,left_index=True, right_index=True)

# use random forest to predict all missing values in AllData
imp = IterativeImputer(estimator=ExtraTreesRegressor(n_estimators=100, random_state=0),missing_values=np.nan,max_iter = 30, random_state=0)
imp.fit(AllData)
A = imp.transform(AllData)

# reshape output back to shape of AllData
X = A[:,0].reshape(len(countries),1)
for i in range(1,len(A[0])):
    B = A[:,i].reshape(len(countries),1)
    X = np.column_stack((X,B))

C:\Users\Lewis\Miniconda3\lib\site-packages\sklearn\impute\_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


**Correlations between features**

In [7]:
# Correlations and heatmap between features of all 3 datasets
IndicatorList = np.array(Indicator)
IndicatorList =  np.concatenate((IndicatorList, IndicatorGen), axis=None)
IndicatorList = np.concatenate((IndicatorList, IndicatorUnicef1), axis=None)
CORR = Correlation_features(X,IndicatorList)
CORRELATION = CORR[0]
SIGNIFICANCE = CORR[1]
significant_correlations(CORRELATION,SIGNIFICANCE,IndicatorList)
correlation_heatmap(CORRELATION)

Indicators with a moderate correlation (>0.5) and are significant at a 5% level of significance:
Adult illiterate population, 15+ years, % female V.S. Youth illiterate population, 15-24 years, % female ---> 0.6071175137926053

Adult illiterate population, 15+ years, % female V.S. Youth illiterate population, 15-24 years, % male ---> -0.6086548386440993

Adult illiterate population, 15+ years, % female V.S. Youth illiterate population, 15-24 years, gender difference (%) ---> 0.6073845030848452

Adult literacy rate, population 15+ years, gender parity index (GPI) V.S. Literacy rate, adult female (% of females ages 15 and above) ---> 0.9488707310686741

Adult literacy rate, population 15+ years, gender parity index (GPI) V.S. Literacy rate, adult gender difference (%) ---> 0.9392786610924295

Adult literacy rate, population 15+ years, gender parity index (GPI) V.S. Literacy rate, adult male (% of males ages 15 and above) ---> 0.8598597531315142

Adult literacy rate, population 15+ years, 

**3D Scatterplots between features of the 3 datasets (Literacy, Political and Unicef data)** 

**INDEX LIST OF FEATURES IN X:**

0 --> 'Adult illiterate population, 15+ years, % female'

1 --> 'Adult literacy rate, population 15+ years, gender parity index (GPI)'

2 --> 'Learning poverty: gender difference (%)'

3 --> 'Learning poverty: Share of Female Children at the End-of-Primary age below minimum reading proficiency adjusted by Out-of-School Children (%)'

4 --> 'Learning poverty: Share of Male Children at the End-of-Primary age below minimum reading proficiency adjusted by Out-of-School Children (%)'

5 --> 'Literacy rate, adult female (% of females ages 15 and above)'

6 --> 'Literacy rate, adult gender difference (%)'

7 --> 'Literacy rate, adult male (% of males ages 15 and above)'

8 --> 'Literacy rate, youth (ages 15-24), gender parity index (GPI)'

9 --> 'Literacy rate, youth female (% of females ages 15-24)'

10 --> 'Literacy rate, youth gender difference (%)'

11 --> 'Literacy rate, youth male (% of males ages 15-24)'

12 --> 'Mean performance on the mathematics scale. Female'

13 --> 'Mean performance on the mathematics scale. Gender Difference'

14 --> 'Mean performance on the mathematics scale. Male'

15 --> 'Mean performance on the reading scale. Female'

16 --> 'Mean performance on the reading scale. Gender Difference'

17 --> 'Mean performance on the reading scale. Male'

18 --> 'Mean performance on the science scale. Female'

19 --> 'Mean performance on the science scale. Gender Difference'

20 --> 'Mean performance on the science scale. Male'

21 --> 'Youth illiterate population, 15-24 years, % female'

22 --> 'Youth illiterate population, 15-24 years, % male'

23 --> 'Youth illiterate population, 15-24 years, gender difference (%)'

24 --> 'Proportion of seats held by women in national parliaments (%)'

25 --> 'Proportion of women in ministerial level positions (%)'

26 --> 'Political Empowerment Score'

27 --> 'Proportion of women in managerial positions'

28 --> 'Proportion of women in senior and middle management positions'

29 --> 'Share of female judges'

30 --> 'Share of female police officers'

31 --> 'Financial inclusion (%) - male'

32 --> 'Financial inclusion (%) - female '

33 --> 'Urban labour force participation rate (%) - male'

34 --> 'Urban labour force participation rate (%) - female'

35 --> 'Rural labour force participation rate (%) - male'

36 --> 'Rural labour force participation rate (%) - female'

37 --> 'Total labour force participation rate (%) - male'

38 --> 'Total labour force participation rate (%) - female'

39 --> 'Demand for family planning satisfied with modern methods (%) - Women aged 15-49'

40 --> 'Demand for family planning satisfied with modern methods (%) - Women aged 15-19'

In [8]:
# SCATTERPLOTS 
# define features to plot and label attribute
Index1 = 11 # First feature 0 to 23
Index2 = 28 # Second feature 24 to 30
Index3 = 31 # Third feature 31 to 40
attribute = 'GNI' #set to 'Region' or 'GNI'

# define regions, gni and colours in plot
r = File.Region.unique()
g = File.GNI.unique()
cl = ['red','blue','green','orange','purple','cyan']

# scaled x and y values to use in scatterplot (Using datset in X)
scaler = StandardScaler()
xvalues = scaler.fit_transform(X[:,Index1].reshape(-1, 1))
yvalues = scaler.fit_transform(X[:,Index2].reshape(-1, 1))
zvalues = scaler.fit_transform(X[:,Index3].reshape(-1, 1))
RCG = get_regions_and_gni(File,r,g,cl,attribute)
REGIONS = RCG[0]
COLOUR = RCG[1]
GNI = RCG[2]

# plot scatter plot
plot_scatter(xvalues,yvalues,zvalues,REGIONS,GNI,attribute,Index1,Index2,Index3)

**Centroid K-Means (3D - one feature from each dataset) - takes hours to run this code**

In [ ]:
# CENTROID K-MEANS OF ALL COMBINATIONS OF FEATURES FROM THE LITERACY, POLITICS AND UNICEF DATASETS (WHERE ONE FEATURE
# FROM EACH DATASET IS SELECTED FOR EVERY COMBINATION)

# initialise number of feature combinations and arrays
alllist = [Indicator,IndicatorGen,IndicatorUnicef1]
n = len(list(itertools.product(*alllist)))
TestAccuracyMean = np.zeros((1,n))
TestFeatures = np.zeros((3,n))
a = 0
for i in range(len(Indicator)):
    for j in range(len(IndicatorGen)):
        for k in range(len(IndicatorUnicef1)):
            TestFeatures[0,a] = i
            TestFeatures[1,a] = int(len(Indicator)) + j
            TestFeatures[2,a] = int(len(Indicator)) + int(len(IndicatorGen)) + k  
            a = a + 1

# choose features for kmeans,initialise variables and compute test accuracy of each combination
N = 50 # number of times each combination of features is run
TestAccuracyMeanList = np.zeros((N,n)) 
RCG = get_regions_and_gni(File,r,g,c,attribute)
REGIONS = RCG[0]
COLOUR = RCG[1]
GNI = RCG[2]
q = g # set q as either g (for GNI) or r (for Region)
OUTPUT = GNI # set as either GNI or REGIONS
t_size = 0.2 # proportion of points in data split into test and training sets
num = 5 # number of different splits of the data into test and training sets
attribute = 'GNI' #set to 'Region' or 'GNI'
scaler = StandardScaler()
for c in range(0,N):
    print(c)
    for b in range(0,n):
        print(b)
        # initialise inputs
        Index1 = int(TestFeatures[0,b]) # First feature 0 to 23
        Index2 = int(TestFeatures[1,b]) # Second feature 24 to 30
        Index3 = int(TestFeatures[2,b]) # Third feature 31 to 40
        InputMatrix = np.column_stack([X[:,Index1],X[:,Index2],X[:,Index3]])
        InputMatrix = scaler.fit_transform(InputMatrix)
        xvalues = InputMatrix[:,0]
        yvalues = InputMatrix[:,1]
        zvalues = InputMatrix[:,2]
        TestMean = K_means_test_accuracy(num,t_size,q,attribute,OUTPUT,xvalues,yvalues,zvalues,TestAccuracyMean,b,c)
    TestAccuracyMeanList[c,:] = TestAccuracyMean

# mean and standard deviation test accuracy for each combination
FinalAccuracyMean= np.mean(TestAccuracyMeanList, axis=0) # mean accuracy of each combination of features
FinalAccuracyStd = np.std(TestAccuracyMeanList, axis=0) # standard dev of test accuracy of each combination of features
print(FinalAccuracyMean)
print(FinalAccuracyStd)

# combination that produces best results:
MAX = np.max(FinalAccuracyMean)
Maxindex = np.argmax(FinalAccuracyMean)
STD = FinalAccuracyStd[int(Maxindex)]
print("\033[1m" + "Features that produce best results using K-Means clustering:"+ "\033[0;0m" + "\n")
print(IndicatorList[int(TestFeatures[0,Maxindex])] + "\n")
print(IndicatorList[int(TestFeatures[1,Maxindex])] + "\n")
print(IndicatorList[int(TestFeatures[2,Maxindex])] + "\n")
print("Mean Test Accuracy --> " + str(MAX) + "\n")
print("std Test Accuracy --> " + str(STD))

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**Top 3 combination of features that produce best results using Centroid K-Means clustering (GNI):**

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**Best:** Literacy rate, youth male (% of males ages 15-24) (11),
Proportion of women in senior and middle management positions (%) (28),
Financial inclusion (%) - male (31)

**(Mean Test Accuracy --> 52.51166667%,                                 Standard Deviation Test Accuracy --> 0.56643759)**

**Second:** Learning poverty: gender difference (%) (2),
Proportion of women in senior and middle management positions (%) (28),
Financial inclusion (%) - male (31)

**(Mean Test Accuracy --> 51.02166667%,                                 Standard Deviation Test Accuracy --> 0.289251)**

**Third:** Literacy rate, youth male (% of males ages 15-24) (11),
Proportion of women in senior and middle management positions (%) (28),
Financial inclusion (%) - female (32) 

**(Mean Test Accuracy --> 50.905%,                                 Standard Deviation Test Accuracy --> 0.5191871)**

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**Top 3 combination of features that produce best results using Centroid K-Means clustering (Region):**

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**Best:** Learning poverty: gender difference (%) (2),
Share of female judges (29),
Financial inclusion (%) - male (31)

**(Mean Test Accuracy --> 45.33833333%,                                 Standard Deviation Test Accuracy --> 1.95110913)**

**Second:** Mean performance on the science scale. Female (18),
Share of female judges (29),
Urban labour force participation rate (%) - female (34)

**(Mean Test Accuracy --> 44.24777778%,                                 Standard Deviation Test Accuracy --> 1.16198585)**

**Third:** Mean performance on the mathematics scale. Male (14),
Share of female judges (29),
Urban labour force participation rate (%) - female (34)

**(Mean Test Accuracy --> 43.87444444%,                                 Standard Deviation Test Accuracy --> 1.79671545)**

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**Centroid K-Means isn't suitable for inferring GNI or Region from features of the datasets (i.e. low test accuracy)**

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

**PCA and logistic regression**

In [12]:
# PCA for feature selection/reduce dimensionality of dataset. Logistic Regression is then used with the features extracted
# from PCA to predict class label of test data

# split dataset into training and test data
OUTPUT = GNI # GNI or REGIONS
X_train, X_test, Y_train, Y_test = train_test_split(X,OUTPUT, test_size=0.3, random_state=0)

# scale data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Principal Component Analysis to find features to use in logistic regression
var = 0.80
pca = PCA(var) # if GNI then best score using 0.8, if REGIONS then best score using >0.92
principalComponents = pca.fit(X_train)
print("\033[1m" + "number of features/components:" + "\033[0;0m")
print(str(pca.n_components_) + "\n")
print("\033[1m" + "variance of data explained by each feature:" + "\033[0;0m")
print(str(pca.explained_variance_ratio_) + "\n")

# logistic regression to predict class labels of X_test
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(X_train, Y_train)
Y_pred = logisticRegr.predict(X_test)
print("\033[1m" + "Test accuracy score using Logistic Regression (0 to 1):" + "\033[0;0m")
print(str(logisticRegr.score(X_test,Y_test)))

# scatter plot of top 3 principal components
scaler.fit(X)
X_train = scaler.transform(X)
pca.fit(X_train)
X_train = pca.transform(X_train)
plot_scatter(X_train[:,0],X_train[:,1],X_train[:,2],REGIONS,GNI,'GNI',Index1,Index2,Index3)

number of features/components:
7

variance of data explained by each feature:
[0.32379655 0.20161977 0.11314837 0.05847708 0.05076459 0.04395451
 0.04332749]

Test accuracy score using Logistic Regression (0 to 1):
0.6111111111111112
